## Bayesian Personalized Ranking (BPR) Algorithm with timestamp and rating information

### Data Loading with sequence 


In [2]:
def DataLoading(filename, splitter):
    """
    Each line of the "user-item-rate-timestamp" file is: userId, itemId, ratingScore, timestamp
    Each element of train is the [[item1, time1], [item2, time2] of the user, sorted by timestamp
    Each element of test  is the [user, item, time] interaction,              sorted by timestamp
    """
    train = []  
    test = []
    
    # load ratings into train
    num_ratings = 0
    num_item = 0

    count = 0

    with open(filename, "r") as f:
        line = f.readline()

        # simple note of progress
        count += 1
        if count%10000 == 0:
            sys.stderr.write("\nCurrently at line %d\n" % count)
            sys.stderr.flush()

        while line != None and line != "":
            arr = line.split(splitter)
            if (len(arr) < 4):
                continue
            user, item, time = int(arr[0]), int(arr[1]), long(arr[3]) 
            if (len(train) <= user):
                train.append([])
            train[user].append([item, time])
            num_ratings += 1
            num_item = max(item, num_item)
            line = f.readline()
    num_user = len(train)
    num_item = num_item + 1
    
    # sort ratings of each user by time
    def getTime(item):
        return item[-1];
    for u in range (len(train)):
        train[u] = sorted(train[u], key = getTime)
    
    # split into train/test
    for u in range (len(train)):
        # for k in range(K):
        if (len(train[u]) < 2):
        # if (len(train[u]) == 0):
            break
        test.append([u, train[u][-1][0], train[u][-1][1]])
        del train[u][-1]    # delete the last element from train
            
    # sort the test ratings by time
    test = sorted(test, key = getTime)
    
    return train, test, num_user, num_item, num_ratings

Import packages needed.

In [17]:
import sys
from MFbpr import MFbpr
import multiprocessing as mp
import os
import pandas as pd
import numpy as np

In [44]:
# #Try loading a sample dataset.
# dataset = 'data/yelp.rating'
# # dataset = '/Users/ito/venv/pyspark-rec/ml-100k/u.data' # need to shift (i.e., -1) on ID
# splitter = "\t"
# train, test, num_user, num_item, num_ratings = DataLoading(dataset, splitter)
# sys.stderr.write("Load data (%s) done.\n" %(dataset))
# sys.stderr.write("#users: %d, #items: %d, #ratings: %d" %(num_user, num_item, num_ratings))

Load .csv transaction data (I am showing Tops as an example, while I am thinking of processing shopping mall data since I hope to have a category information that is already "provided" in shopping mall product_master.csv)

In [21]:
# ShoppingMall = pd.read_csv('/Users/ito/venv/CG-RecoEng/data/SM-focus.csv', sep=',', engine='c')
# data = ShoppingMall
Tops_trans_df = pd.read_csv('/Users/ito/venv/CG-RecoEng/data/og_data/Tops_supermarket.csv', 
                            parse_dates=['TransactionDate'], sep=',', engine='c')
data = Tops_trans_df

In [19]:
print Tops_trans_df.shape
Tops_trans_df.head(2)

(1995491, 13)


,TypeGroup,BUID,BranchID,TransactionDate,CustomerID,CardNo,TicketNumber,SKUCode,Spending,DeptCode,SubDeptCode,QTY,tDate2
0,1,150,4294,2017-02-27 20:12:00,312CB58B-09B6-49CA-9F1D-AEF8D56E3F4E,8035450104,40155510,8851954101211,17.00,1.0,3.0,1,2017-02-27
1,1,150,2865,2017-01-18 12:28:00,35FFAE65-80FA-4A2B-8F05-98ACAE82EF86,8028520371,80176812,8851717901607,100.39,1.0,4.0,2,2017-01-18


Yup, the file below (ProductMaster is with category information while ProductMaster_Tops is with details (except pricing info which we concerned))

In [20]:
# productfile1 = '/Users/ito/venv/CG-RecoEng/data/og_data/ProductMaster.csv'
# product1 = pd.read_csv(productfile1, sep=',', engine='c')
productfile2 = '/Users/ito/venv/CG-RecoEng/data/og_data/ProductMaster_Tops.csv'
product2 = pd.read_csv(productfile2, sep=',', engine='c')

Data Wrangling to the certain tuple based format with timestamps.

In [22]:
# Creating the data that follows the timestamp format
# type(data.TransactionDate[0])
data['OrdinalTime'] = data.TransactionDate.map(pd._libs.tslib.Timestamp.toordinal)

idx = pd.Categorical(list(data.CustomerID)).codes
itm = pd.Categorical(list(data.SKUCode)).codes

data['CustomerCat'] = idx #+1
data['ItemCat'] = itm #+1
sub_data = data[['CustomerCat','ItemCat','QTY','OrdinalTime']]

print sub_data.shape
sub_data.head()

(1995491, 4)


,CustomerCat,ItemCat,QTY,OrdinalTime
0,1785,31637,1,736387
1,1971,30314,2,736347
2,7656,40054,1,736398
3,595,25826,1,736363
4,6470,13776,1,736375


In [23]:
sub_data = sub_data.sort_values(by=['CustomerCat','ItemCat'])
# sub_data.to_csv('Tops_user-item_data', index=False,header=False, sep='\t')
sub_data.to_csv('Tops_BPR_2_data.csv', index=False,header=False)

Match the ID and the "series number" by zipping them

In [25]:
idx_match = zip(data.CustomerID, data['CustomerCat'] )
itm_match = zip(data.SKUCode, data['ItemCat'])

In [26]:
idx_match[:5]

[('312CB58B-09B6-49CA-9F1D-AEF8D56E3F4E', 1785),
 ('35FFAE65-80FA-4A2B-8F05-98ACAE82EF86', 1971),
 ('D2F9C8EA-51C7-4824-B7F5-9ADAAE665EDB', 7656),
 ('0FEF3D2E-DFF0-4CAF-8CB2-471FF2404348', 595),
 ('B1E1C8CF-D484-4141-89A5-D24C570EF70A', 6470)]

In [27]:
itm_match[:5]

[(8851954101211, 31637),
 (8851717901607, 30314),
 (8858935387008, 40054),
 (8850329315543, 25826),
 (4005808695829, 13776)]

Prepare matching lists (saved to files also).

In [28]:
itm_match.sort()
print itm_match.__len__()
itm_final = set(itm_match)
print itm_final.__len__()

1995491
44057


In [29]:
idx_match.sort()
print idx_match.__len__()
idx_final = set(idx_match)
print idx_final.__len__()

1995491
9204


In [30]:
IDX = list(idx_final)
ITM = list(itm_final)
IDX.sort()
ITM.sort()

In [31]:
pd.DataFrame(IDX).to_csv('user_matching_list', index=False,header=False, sep='\t')
pd.DataFrame(ITM).to_csv('item_matching_list', index=False,header=False, sep='\t')

## Data Preparation

In [32]:
dataset = 'Tops_BPR_2_data.csv'
splitter = ","
train, test, num_user, num_item, num_ratings = DataLoading(dataset, splitter)
sys.stderr.write("Load data (%s) done." %(dataset))
sys.stderr.write("#users: %d, #items: %d, #ratings: %d" %(num_user, num_item, num_ratings))

Load data (Tops_BPR_2_data.csv) done.#users: 9204, #items: 44057, #ratings: 1995491

Setup the parameters for bpr with multiprocessing.

In [33]:
factors = 10
learning_rate = 0.01
reg = 0.01
init_mean = 0
init_stdev = 0.01
maxIter = 50
num_thread = mp.cpu_count()

In [34]:
bpr = MFbpr(train, test, num_user, num_item, factors, learning_rate, reg, init_mean, init_stdev)

In [36]:
import time
starting = time.time()

# bpr.build_model(maxIter, num_thread, batch_size=32)
bpr.build_model(maxIter, num_thread, batch_size=1000)

ending = time.time()
print 'time elapsed: ' + str(ending - starting)

num of user = 9204
Training MF-BPR with: learning_rate=0.01, regularization=0.0100, factors=10, #epoch=50, batch_size=1000.
Iter=0 [59.5 s] HitRatio@100 = 0.017, NDCG@100 = 0.004 [0.5 s]
Iter=1 [57.4 s] HitRatio@100 = 0.045, NDCG@100 = 0.008 [0.6 s]
Iter=2 [57.6 s] HitRatio@100 = 0.049, NDCG@100 = 0.010 [0.8 s]
Iter=3 [57.0 s] HitRatio@100 = 0.056, NDCG@100 = 0.011 [0.8 s]
Iter=4 [54.6 s] HitRatio@100 = 0.059, NDCG@100 = 0.013 [0.8 s]
Iter=5 [56.1 s] HitRatio@100 = 0.059, NDCG@100 = 0.014 [0.9 s]
Iter=6 [57.0 s] HitRatio@100 = 0.059, NDCG@100 = 0.014 [0.8 s]
Iter=7 [57.1 s] HitRatio@100 = 0.059, NDCG@100 = 0.012 [0.8 s]
Iter=8 [55.8 s] HitRatio@100 = 0.063, NDCG@100 = 0.013 [0.9 s]
Iter=9 [56.9 s] HitRatio@100 = 0.063, NDCG@100 = 0.013 [0.9 s]
Iter=10 [57.0 s] HitRatio@100 = 0.066, NDCG@100 = 0.014 [0.8 s]
Iter=11 [57.0 s] HitRatio@100 = 0.052, NDCG@100 = 0.011 [0.8 s]
Iter=12 [57.0 s] HitRatio@100 = 0.073, NDCG@100 = 0.014 [0.9 s]
Iter=13 [57.0 s] HitRatio@100 = 0.063, NDCG@100 = 0.01

Example of single prediction -- affinity(user, item)

In [37]:
bpr.predict(test[0][0],test[0][1])

2.7778526114505753

Example of multi prediction -- affinity(user, items)

In [38]:
bpr.predictions(test[0][0])

array([-1.88487989, -1.76849832, -1.89922613, ..., -1.59746948,
       -1.72308219, -1.81282518])

#### Return the top 5 item of interest for each customer

In [59]:
import heapq

res = []
top_howmany = 5
for k in range(num_user):
    temp_pred = bpr.predictions(k)
    top_list = heapq.nlargest(top_howmany,temp_pred)
    res.append([i for i in range(len(temp_pred)) if temp_pred[i] in top_list])

In [68]:
itm_match = zip(data['ItemCat'],data.SKUCode)
itm_dict = dict(itm_match)

In [87]:
res_df = pd.DataFrame(res)

Translate category ID to SKUID.

In [103]:
def return_SKUID(input):
    return [itm_dict[input[0]], itm_dict[input[1]], itm_dict[input[2]], itm_dict[input[3]], itm_dict[input[4]]]

res_SKUID = res_df.apply(return_SKUID, axis=1)

In [106]:
sku_match = zip(product2.SKUID,product2.PRODUCT_ENG_DESC)
sku_dict = dict(sku_match)

In [107]:
def return_desc(input):
    return [sku_dict[input[0]], sku_dict[input[1]], sku_dict[input[2]], sku_dict[input[3]], sku_dict[input[4]]]

res_desc = res_SKUID.apply(return_desc, axis=1)

In [109]:
res_desc.shape

(9204, 5)

In [55]:
# print(heapq.nlargest(3,nums))
lst = [0,1,3,11,0]
some = [i for i in range(len(lst)) if lst[i] in heapq.nlargest(3,lst)]

In [110]:
cstmfile = '/Users/ito/venv/CG-RecoEng/data/og_data/Customer_Profile.csv'
customer = pd.read_csv(cstmfile, sep=',', engine='c')

def strip_brace(input):
#     input = input.strip('{')
#     input = input.strip('}')
    return input[1:-1]
#     return input

customer.CustomerID = customer.CustomerID.map(strip_brace)

In [131]:
customer_sorted = customer.sort_values(by='CustomerID')
customer_sorted = customer_sorted.reset_index()
customer_sub = customer_sorted[['CustomerID','DateofBirth','Nationality','Gender','MaritalStatus','NoofChildren','HaveOnlineaccount']]

In [132]:
customer_sub
Reco_result = pd.concat([customer_sub, res_desc], axis=1)

In [133]:
Reco_result.to_csv('Reco_result.csv', index=False)

## Appendix
### Efficiency test

In [ ]:
@exeTime
def predict1(bpr, u, i):
    res = np.inner(bpr.U_np[u], bpr.V_np[i])
    return res

@exeTime
def predict2(bpr, u, i):
    res = T.dot(bpr.U[u], bpr.V[i]).eval()
    return res

In [ ]:
def exeTime(func):
    def newFunc(*args, **args2):
        t0 = time.time()
        print "@%s, {%s} start" % (time.strftime("%X", time.localtime()), func.__name__)
        back = func(*args, **args2)
        print "@%s, {%s} end" % (time.strftime("%X", time.localtime()), func.__name__)
        print "@%.3fs taken for {%s}" % (time.time() - t0, func.__name__)
        return back
    return newFunc

In [ ]:
alist = np.linspace(1,100,10000000)

%%cython
def c_add(int a, int b):
    cdef int s = a+b
    return s

def p_add(a,b):
    return a+b

%%cython
def cython_mean(double[:] x):
    cdef double total = 0
    for i in range(len(x)):
        total += x[i]
    return total / len(x)

def python_mean(xs):
    total = 0
    for i in range(len(xs)):
        total += xs[i]
    return total/len(xs)

from numba import jit
@jit
def numba_mean(xs):
    total = 0
    for i in range(len(xs)):
        total += xs[i]
    return total/len(xs)


%timeit python_mean(alist)
%timeit cython_mean(alist)

%timeit np.mean(alist)

%timeit numba_mean(alist)

### Reducing mem for data frame

In [ ]:
sub_data.info()

In [ ]:
sub_data.QTY = sub_data.QTY.astype(np.int16)
sub_data.OrdinalTime = sub_data.OrdinalTime.astype(np.int32)
sub_data.info()